In [1]:
library(tidyverse)
library(here)
library(distrEx)

data_path <- here('notebooks/1_hellinger_distance/data')
figure_path <- here('figures')

# make folder
ifelse(!dir.exists(file.path(figure_path)), dir.create(file.path(figure_path)), FALSE)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.6      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

here() starts at /home/wonseok/2022_DATA_SYNTHESIS

Loading required package: distr

Loading required package: startupmsg

Utilities for Start-Up Messages (version 0.9.6)

For more information see ?"startupmsg", NEWS("startupmsg")


Loading required package: sfsmisc


Attaching package: ‘sfsmisc’


The following object is masked from ‘package:dplyr’:

    last


Object Oriented Implementation of Distributions (version 2.9.0)

Attention: Arithmetics on distribution objects are understood as operations on corresponding random variables (r.v.s); see distrARITH().
Some functions from package 'stats' are

[1] FALSE

In [2]:
D0 <- read_csv(here(data_path, "D0.csv"))
S01 <- read_csv(here(data_path, "S1_0.1.csv"))
S1 <- read_csv(here(data_path, "S1_1.csv"))

Warning message:
“One or more parsing issues, see `problems()` for details”
Rows: 1109646 Columns: 87
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (3): PTH_BPSY_BPTH_BPSY_RSLT_CONT, PT_BSNF_BSPT_FRST_DIAG_CD, PT_BSNF_B...
dbl (78): PT_SBST_NO, TIME, DG_RCNF_RLPS, EX_DIAG_ALT(GPT), EX_DIAG_AST(GOT)...
lgl  (6): PTH_MNTY_IMPT_HM1E_RSLT_CD, PTH_MNTY_IMPT_HP2E_RSLT_CD, PTH_MNTY_I...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message:
“One or more parsing issues, see `problems()` for details”
Rows: 114900 Columns: 87
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (3): PTH_BPSY_BPTH_BPSY_RSLT_CONT, PT_BSNF_BSPT_FRST_DIAG_CD, PT_BSNF_...
dbl  (81): PT_SBST_NO, DG_RCNF_RLPS, EX_DIAG_ALT(GPT), EX_DIAG_AST(GOT), EX_...
lgl   (2): PTH_MNTY_IMPT_HP2E_RSLT_CD, PTH_SRGC_SG

In [7]:
D0 %>% filter(TIME == 0)

PT_SBST_NO,TIME,DG_RCNF_RLPS,EX_DIAG_ALT(GPT),EX_DIAG_AST(GOT),EX_DIAG_Albumin,EX_DIAG_Alk. Phos,EX_DIAG_BUN,EX_DIAG_Basophil(#),EX_DIAG_Basophil(%),⋯,PT_BSNF_BSPT_N_STAG_VL,PT_BSNF_BSPT_SEX_CD,PT_BSNF_BSPT_STAG_VL,PT_BSNF_BSPT_T_STAG_VL,PT_BSNF_BSPT_FRST_DIAG_YMD,TRTM_CASB_CSTR_NT,TRTM_CASB_CSTR_PRPS_CD,TRTM_CASB_CSTR_REGN_CD,TRTM_RD_RDT,DEAD_NFRM_DEAD
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,0,60,25,4.4,65,15.5,0.03,0.4,⋯,1,F,3,3,1,NA,NA,NA,NA,0
2,0,0,NA,NA,NA,NA,NA,NA,NA,⋯,0,M,2,3,1,NA,NA,NA,NA,0
3,0,0,NA,NA,NA,NA,NA,NA,NA,⋯,2,M,4,4,1,NA,NA,NA,NA,0
4,0,0,NA,NA,NA,NA,NA,NA,NA,⋯,NA,M,4,3,1,NA,NA,NA,NA,0
6,0,0,NA,NA,NA,NA,NA,NA,NA,⋯,NA,M,NA,NA,1,NA,NA,NA,NA,0
7,0,0,NA,NA,NA,NA,NA,NA,NA,⋯,2,F,4,4,1,NA,NA,NA,NA,0
8,0,0,NA,NA,NA,NA,NA,NA,NA,⋯,0,M,2,3,1,NA,NA,NA,NA,0
9,0,0,NA,NA,NA,NA,NA,NA,NA,⋯,2,M,4,3,1,NA,NA,NA,NA,0
10,0,0,NA,NA,NA,NA,NA,NA,NA,⋯,2,M,3,3,1,NA,NA,NA,NA,0


In [3]:
column_names <- colnames(D0)
basic_columns <- column_names[grepl("PT_BSNF",column_names)]

In [4]:
d0 <- D0[c("PT_SBST_NO",'TIME',basic_columns)] %>%
    filter(TIME == 0) %>%
    mutate(TYPE = 'original')

s01 <- S01[c("PT_SBST_NO",'TIME',basic_columns)]  %>%
    filter(TIME == "2222-02-22") %>% 
    mutate(TYPE = 'epsilon 0.1',
          TIME = 0)

s1 <- S1[c("PT_SBST_NO",'TIME',basic_columns)]  %>%
    filter(TIME == "2222-02-22") %>% 
    mutate(TYPE = 'epsilon 1',
            TIME = 0)

In [5]:
whole_data <- rbind(d0, s01, s1)

## Analyze the whole data
- 엡실론별로 어떻게 분포가 변하는지 (성별, age, first_diagnosis_ymd)
- Hellinger Distance

In [6]:
library(ggplot2)
library(sm)

Package 'sm', version 2.2-5.7: type help(sm) for summary information



Hellinger Distance

In [7]:
ori <- whole_data %>% 
    filter(TYPE == 'original') %>%
    select(PT_BSNF_BSPT_IDGN_AGE) 

s01_vector <- whole_data %>% 
    filter(TYPE == 'epsilon 0.1') %>%
    select(PT_BSNF_BSPT_IDGN_AGE) 

s1_vector <- whole_data %>% 
    filter(TYPE == 'epsilon 1') %>%
    select(PT_BSNF_BSPT_IDGN_AGE) 

In [22]:
ori <- ori %>% unlist() %>% as.vector()
s01_vector <- s01_vector %>% unlist() %>% as.vector()
s1_vector <- s1_vector %>% unlist() %>% as.vector()

In [41]:
HD <- function(x, y){
    hd <- HellingerDist(DiscreteDistribution(x), DiscreteDistribution(y))
    return(hd[['Hellinger distance']])
}

In [43]:
HD(ori, s01_vector)

[1] 0.1101243